# Coding Block 2 - Interpretability with SHAP

### Load the packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import shap
'''
...
'''

### Read the dataset 
You can also compare processed and non-processed data.

In [ ]:
diab=pd.read_csv('diabetes.csv')
diab_cleaned=pd.read_csv('diabetes_cleaned.csv')

### Copy the code from your last successful classifiers (RF, XGBoost, ...)

In [ ]:

datasets = {
    "Original Dataset": diab,
    "Cleaned Dataset": diab_cleaned
}

for dataset_name, dataset in datasets.items():
    print(f"\n{'='*80}")
    print(f"Analysis for {dataset_name}")
    print(f"{'='*80}")
    
    # Split features and target
    X = dataset.iloc[:, :-1]
    y = dataset.iloc[:, -1]
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # (i) Train a Random Forest model
    print("\n--- Random Forest Model ---")
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_model.fit(X_train, y_train)
    
    # (ii) Evaluate with classification report
    y_pred_rf = rf_model.predict(X_test)
    print("\nClassification Report for Random Forest:")
    print(classification_report(y_test, y_pred_rf))
    
    # (iii) Print feature importances
    feature_importances = pd.DataFrame({
        'Feature': X.columns,
        'Importance': rf_model.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    print("\nFeature Importances for Random Forest:")
    print(feature_importances)
    
    # Plot feature importances
    plt.figure(figsize=(10, 6))
    plt.barh(feature_importances['Feature'], feature_importances['Importance'])
    plt.xlabel('Importance')
    plt.title(f'Random Forest Feature Importances - {dataset_name}')
    plt.gca().invert_yaxis()  # Display the highest importance at the top
    plt.tight_layout()
    plt.show()
    
    # (iv) Train XGBoost model
    print("\n--- XGBoost Model ---")
    xgb_model = xgb.XGBClassifier(n_estimators=100, random_state=42)
    xgb_model.fit(X_train, y_train)
    
    # Evaluate XGBoost
    y_pred_xgb = xgb_model.predict(X_test)
    print("\nClassification Report for XGBoost:")
    print(classification_report(y_test, y_pred_xgb))
    
    # XGBoost feature importances
    xgb_importances = pd.DataFrame({
        'Feature': X.columns,
        'Importance': xgb_model.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    print("\nFeature Importances for XGBoost:")
    print(xgb_importances)
    
    # Plot XGBoost feature importances
    plt.figure(figsize=(10, 6))
    plt.barh(xgb_importances['Feature'], xgb_importances['Importance'])
    plt.xlabel('Importance')
    plt.title(f'XGBoost Feature Importances - {dataset_name}')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
    
    # Compare the models
    print("\n--- Model Comparison ---")
    print(f"Random Forest Accuracy: {rf_model.score(X_test, y_test):.4f}")
    print(f"XGBoost Accuracy: {xgb_model.score(X_test, y_test):.4f}")
    
    # Check if there are differences in the predictions
    disagreements = np.sum(y_pred_rf != y_pred_xgb)
    print(f"Number of disagreements between models: {disagreements} out of {len(y_test)} samples")

### Create a SHAP summary plot that provides an overview of the average feature importance

In [ ]:
### Create a SHAP summary plot that provides an overview of the average feature importance

import shap
import matplotlib.pyplot as plt

# Create a figure with two subplots
plt.figure(figsize=(20, 10))

# XGBoost model SHAP summary plot
plt.subplot(1, 2, 1)
explainer = shap.TreeExplainer(xgb_model)
shap_values = explainer.shap_values(X_test)
shap.summary_plot(shap_values, X_test, feature_names=X.columns, show=False)
plt.title("XGBoost Feature Importance", fontsize=15)
plt.tight_layout()

# Random Forest model SHAP summary plot
plt.subplot(1, 2, 2)
explainer = shap.TreeExplainer(rf_model)
shap_values = explainer.shap_values(X_test)
if isinstance(shap_values, list):  # For multi-class output
    shap_values = shap_values[1]  # For binary classification, use class 1
shap.summary_plot(shap_values, X_test, feature_names=X.columns, show=False)
plt.title("Random Forest Feature Importance", fontsize=15)

plt.tight_layout()
plt.show()

### Create SHAP waterfall plots that describe the model prediction for one or two individuals from the test dataset

In [ ]:
### Create SHAP waterfall plots that describe the model prediction for one or two individuals from the test dataset

import shap
import matplotlib.pyplot as plt

# Choose two individuals from the test dataset
individual_indices = [0, 5]  # You can change these indices if needed

# Create a figure with 2 rows and 2 columns
fig, axs = plt.subplots(2, 2, figsize=(20, 16))

# For XGBoost Model
explainer_xgb = shap.TreeExplainer(xgb_model)
shap_values_xgb = explainer_xgb.shap_values(X_test)
expected_value_xgb = explainer_xgb.expected_value

# For Random Forest Model
explainer_rf = shap.TreeExplainer(rf_model)
shap_values_rf = explainer_rf.shap_values(X_test)
# Handle different return formats for Random Forest models
if isinstance(shap_values_rf, list):  # For multi-class output
    expected_value_rf = explainer_rf.expected_value[1]  # Class 1 expected value
    shap_values_rf = shap_values_rf[1]  # Class 1 SHAP values
else:
    expected_value_rf = explainer_rf.expected_value

# Create waterfall plots for each individual and model
for i, idx in enumerate(individual_indices):
    # XGBoost waterfall plot
    plt.sca(axs[i, 0])
    shap.waterfall_plot(shap.Explanation(
        values=shap_values_xgb[idx], 
        base_values=expected_value_xgb, 
        data=X_test.iloc[idx], 
        feature_names=X_test.columns.tolist()
    ), show=False)
    plt.title(f"XGBoost - Individual {idx}", fontsize=14)
    
    # Random Forest waterfall plot
    plt.sca(axs[i, 1])
    shap.waterfall_plot(shap.Explanation(
        values=shap_values_rf[idx], 
        base_values=expected_value_rf, 
        data=X_test.iloc[idx], 
        feature_names=X_test.columns.tolist()
    ), show=False)
    plt.title(f"Random Forest - Individual {idx}", fontsize=14)

plt.tight_layout()
plt.show()

# Print the actual predictions for these individuals
print(f"Individual {individual_indices[0]} - XGBoost prediction: {xgb_model.predict_proba(X_test.iloc[[individual_indices[0]]])[:, 1][0]:.4f}, "
      f"Actual: {y_test.iloc[individual_indices[0]]}")
print(f"Individual {individual_indices[0]} - Random Forest prediction: {rf_model.predict_proba(X_test.iloc[[individual_indices[0]]])[:, 1][0]:.4f}, "
      f"Actual: {y_test.iloc[individual_indices[0]]}")
print(f"Individual {individual_indices[1]} - XGBoost prediction: {xgb_model.predict_proba(X_test.iloc[[individual_indices[1]]])[:, 1][0]:.4f}, "
      f"Actual: {y_test.iloc[individual_indices[1]]}")
print(f"Individual {individual_indices[1]} - Random Forest prediction: {rf_model.predict_proba(X_test.iloc[[individual_indices[1]]])[:, 1][0]:.4f}, "
      f"Actual: {y_test.iloc[individual_indices[1]]}")